# Physics engines

Gazebo has interfaces with four physics engines

* ODE
* Simbody
* Bullet
* DART

from which only DART has to be compiled separately.
The modules presented below allow the generation of the necessary physics engine's SDF parameters using classes.

## Physics engine object

The physics engine object holds the global parameters valid for all physics engines. 

If this modules is used to generate the SDF data, the default parameters of the physics engine chosen (`ode`, `bullet`, `simbody` or `dart`) will be used.

In [1]:
from pcg_gazebo.simulation.physics import Physics
physics = Physics()

# Iterate through all parameters
for name in physics.get_parameter_names():
    print('{}: {}'.format(name, physics.get_parameter(name)))
    

default: False
max_contacts: 20
name: default_physics
real_time_update_rate: 1000
max_step_size: 0.001
engine: ode
real_time_factor: 1


In [2]:
# The description of the parameters is also available
for name in physics.get_parameter_names():
    physics.print_description(name)

default
  Description: If true, this physics element is set as the default physics profile for the world. If multiple default physics elements exist, the first element marked as default is chosen. If no default physics element exists, the first physics element is chosen.
  Current value: False
max_contacts
  Description: Maximum number of contactsallowed between two entities. This value can be over ridden by a max_contacts element in a collision element
  Current value: 20
name
  Description: The name of this set of physics parameters
  Current value: default_physics
real_time_update_rate
  Description: Rate at which to update the physics engine (UpdatePhysics calls per real-time second)
  Current value: 1000
max_step_size
  Description: Maximum time step size at which every system in simulation can interact with the states of the world
  Current value: 0.001
engine
  Description: The type of the dynamics engine. Current options are ode, bullet, simbody and dart. Defaults to ode if lef

In [3]:
# It is also possible to generate the SDF files.
# The SDF object can also be retrieved and altered if necessary
print(physics.to_sdf('physics'))


<physics default="1" name="default_physics" type="ode">
  <max_contacts>20</max_contacts>
  <real_time_update_rate>1000.0</real_time_update_rate>
  <max_step_size>0.001</max_step_size>
  <real_time_factor>1.0</real_time_factor>
</physics>



In [4]:
print(physics.to_sdf('world'))

<world name="default">
  <gravity>0 0 -9.8</gravity>
  <include>
    <uri>model://ground_plane</uri>
  </include>
  <include>
    <uri>model://sun</uri>
  </include>
  <physics default="1" name="default_physics" type="ode">
    <max_contacts>20</max_contacts>
    <real_time_factor>1.0</real_time_factor>
    <max_step_size>0.001</max_step_size>
    <real_time_update_rate>1000.0</real_time_update_rate>
  </physics>
  <atmosphere type="adiabatic"/>
  <wind>
    <linear_velocity>0 0 0</linear_velocity>
  </wind>
</world>



In [5]:
# Let's make a custom parameter set for the physics engine
physics.max_step_size = 0.01
physics.real_time_factor = 1
physics.real_time_update_rate = 500
physics.max_contacts = 5
physics.name = 'custom_physics'

print(physics.to_sdf())

<physics default="1" name="custom_physics" type="ode">
  <max_contacts>5</max_contacts>
  <real_time_update_rate>500.0</real_time_update_rate>
  <max_step_size>0.01</max_step_size>
  <real_time_factor>1.0</real_time_factor>
</physics>



In [6]:
# The resulting world file can be exported to an .world file and run in Gazebo
# This shows how to create an SDF file from stratch
sdf = physics.to_sdf('sdf')
print(sdf)

<sdf version="1.6">
  <world name="default">
    <gravity>0 0 -9.8</gravity>
    <include>
      <uri>model://ground_plane</uri>
    </include>
    <include>
      <uri>model://sun</uri>
    </include>
    <physics default="1" name="custom_physics" type="ode">
      <max_contacts>5</max_contacts>
      <real_time_update_rate>500.0</real_time_update_rate>
      <max_step_size>0.01</max_step_size>
      <real_time_factor>1.0</real_time_factor>
    </physics>
    <atmosphere type="adiabatic"/>
    <wind>
      <linear_velocity>0 0 0</linear_velocity>
    </wind>
  </world>
</sdf>



In [7]:
# Export to a .world file
world_filename = '/tmp/physics.world'
sdf.export_xml(filename=world_filename)

In [11]:
# Start a PCG server to run the world file in Gazebo
from pcg_gazebo.task_manager import Server
server = Server()
# Create a simulation manager named default
server.create_simulation('default')
simulation = server.get_simulation('default')
# Run an instance of the empty.world scenario
# This is equivalent to run
#      roslaunch gazebo_ros empty_world.launch
# with the parameters provided to run the world file created
simulation.create_gazebo_task(
    name='gazebo',
    world=world_filename,
    gui=True,
    physics='ode',
    paused=False,
    required=True,
    process_timeout=100)
# A task named 'gazebo' the added to the tasks list
print(simulation.get_task_list())
# But it is still not running
print('Is Gazebo running: {}'.format(simulation.is_task_running('gazebo')))
# Run Gazebo
simulation.run_all_tasks()

2020-09-11 14:19:37,767 | WARNING | __init__ | Error testing Gazebo server, message=Command '['rostopic', 'list']' returned non-zero exit status 1
['gazebo']
Is Gazebo running: False
2020-09-11 14:19:38,030 | WARNING | __init__ | Error testing roscore, message=Command '['rostopic', 'list']' returned non-zero exit status 1
2020-09-11 14:19:38,269 | WARNING | __init__ | Error testing roscore, message=Command '['rostopic', 'list']' returned non-zero exit status 1
2020-09-11 14:19:38,523 | WARNING | __init__ | Error testing roscore, message=Command '['rostopic', 'list']' returned non-zero exit status 1


In [12]:
# Check if the parameters were initialized correctly
# We need a Gazebo proxy object to check it
from pcg_gazebo.task_manager import GazeboProxy
# Create a Gazebo proxy
gazebo_proxy = simulation.get_gazebo_proxy()

# It is important to note that the default get_physics_properties service from Gazebo
# returns only the global and the ODE engine parameters
print(gazebo_proxy.get_physics_properties())

time_step: 0.01
pause: False
max_update_rate: 500.0
gravity: 
  x: 0.0
  y: 0.0
  z: -9.8
ode_config: 
  auto_disable_bodies: False
  sor_pgs_precon_iters: 0
  sor_pgs_iters: 50
  sor_pgs_w: 1.3
  sor_pgs_rms_error_tol: 0.0
  contact_surface_layer: 0.001
  contact_max_correcting_vel: 100.0
  cfm: 0.0
  erp: 0.2
  max_contacts: 5
success: True
status_message: "GetPhysicsProperties: got properties"


In [13]:
simulation.wait()
simulation.kill_all_tasks()

2020-09-11 14:21:19,295 | WARNING | task | Task <gazebo> - Killing process tree...
2020-09-11 14:21:19,341 | WARNING | task | Sending SIGINT to child process id=13469
2020-09-11 14:21:19,343 | WARNING | task | Child process 13469 still running
2020-09-11 14:21:19,346 | WARNING | task | Sending SIGINT to child process id=13487
2020-09-11 14:21:19,348 | WARNING | task | Child process 13487 still running
2020-09-11 14:21:19,350 | WARNING | task | Sending SIGINT to child process id=13492
2020-09-11 14:21:19,352 | WARNING | task | Child process 13492 still running
2020-09-11 14:21:19,353 | WARNING | task | Sending SIGINT to child process id=13561
2020-09-11 14:21:19,354 | WARNING | task | Child process 13561 still running
2020-09-11 14:21:19,355 | WARNING | task | Sending SIGINT to child process id=13555
2020-09-11 14:21:19,357 | WARNING | task | Child process 13555 still running
2020-09-11 14:21:19,358 | WARNING | task | Sending SIGINT to child process id=13468
2020-09-11 14:21:19,359 | WA

## ODE

It is possible to create an instance of the physics engine configuration for each engine available.

The ODE is presented on the following sections.

In [14]:
from pcg_gazebo.simulation.physics import ODE
physics = ODE()

# Iterate through all parameters
for name in physics.get_parameter_names():
    print('{}: {}'.format(name, physics.get_parameter(name)))

line 92.
min_step_size: 0.0001
sor: 1.3
default: False
precon_iters: 0
max_contacts: 20
name: default_physics
contact_max_correcting_vel: 100
engine: ode
real_time_update_rate: 1000
cfm: 0
iters: 50
contact_surface_layer: 0.001
friction_model: pyramid_model
erp: 0.2
use_dynamic_moi_rescaling: False
type: quick
real_time_factor: 1
max_step_size: 0.001


In [15]:
# The description of the parameters is also available
for name in physics.get_parameter_names():
    physics.print_description(name)

min_step_size
  Description: The time duration which advances with each iteration of the dynamics engine, this has to be no bigger than max_step_size under physics block. If left unspecified, min_step_size defaults to max_step_size
  Current value: 0.0001
sor
  Description: Set the successive over-relaxation parameter.
  Current value: 1.3
default
  Description: If true, this physics element is set as the default physics profile for the world. If multiple default physics elements exist, the first element marked as default is chosen. If no default physics element exists, the first physics element is chosen.
  Current value: False
precon_iters
  Description: Experimental parameter
  Current value: 0
max_contacts
  Description: Maximum number of contactsallowed between two entities. This value can be over ridden by a max_contacts element in a collision element
  Current value: 20
name
  Description: The name of this set of physics parameters
  Current value: default_physics
contact_max_co

In [16]:
# It is also possible to generate the SDF files.
# The SDF object can also be retrieved and altered if necessary
print(physics.to_sdf('physics'))

<physics default="1" name="default_physics" type="ode">
  <max_contacts>20</max_contacts>
  <ode>
    <constraints>
      <cfm>0.0</cfm>
      <erp>0.2</erp>
      <contact_surface_layer>0.001</contact_surface_layer>
      <contact_max_correcting_vel>100.0</contact_max_correcting_vel>
    </constraints>
    <solver>
      <min_step_size>0.0001</min_step_size>
      <precon_iters>0</precon_iters>
      <sor>1.3</sor>
      <use_dynamic_moi_rescaling>0</use_dynamic_moi_rescaling>
      <friction_model>pyramid_model</friction_model>
      <type>quick</type>
      <iters>50</iters>
    </solver>
  </ode>
  <max_step_size>0.001</max_step_size>
  <real_time_update_rate>1000.0</real_time_update_rate>
  <real_time_factor>1.0</real_time_factor>
</physics>



In [17]:
print(physics.to_sdf('world'))

<world name="default">
  <gravity>0 0 -9.8</gravity>
  <include>
    <uri>model://ground_plane</uri>
  </include>
  <include>
    <uri>model://sun</uri>
  </include>
  <physics default="1" name="default_physics" type="ode">
    <max_contacts>20</max_contacts>
    <real_time_factor>1.0</real_time_factor>
    <ode>
      <constraints>
        <cfm>0.0</cfm>
        <erp>0.2</erp>
        <contact_surface_layer>0.001</contact_surface_layer>
        <contact_max_correcting_vel>100.0</contact_max_correcting_vel>
      </constraints>
      <solver>
        <min_step_size>0.0001</min_step_size>
        <precon_iters>0</precon_iters>
        <sor>1.3</sor>
        <type>quick</type>
        <friction_model>pyramid_model</friction_model>
        <use_dynamic_moi_rescaling>0</use_dynamic_moi_rescaling>
        <iters>50</iters>
      </solver>
    </ode>
    <max_step_size>0.001</max_step_size>
    <real_time_update_rate>1000.0</real_time_update_rate>
  </physics>
  <atmosphere type="adiabatic"/

In [18]:
# Let's change some parameters
physics.max_step_size = 0.005
physics.friction_model = 'box_model'
physics.sor = 1.5
physics.max_contacts = 10
physics.name = 'custom_ode'
print(physics.to_sdf())

<physics default="1" name="custom_ode" type="ode">
  <max_contacts>10</max_contacts>
  <ode>
    <constraints>
      <cfm>0.0</cfm>
      <erp>0.2</erp>
      <contact_surface_layer>0.001</contact_surface_layer>
      <contact_max_correcting_vel>100.0</contact_max_correcting_vel>
    </constraints>
    <solver>
      <min_step_size>0.0001</min_step_size>
      <precon_iters>0</precon_iters>
      <sor>1.5</sor>
      <use_dynamic_moi_rescaling>0</use_dynamic_moi_rescaling>
      <friction_model>box_model</friction_model>
      <type>quick</type>
      <iters>50</iters>
    </solver>
  </ode>
  <max_step_size>0.005</max_step_size>
  <real_time_update_rate>1000.0</real_time_update_rate>
  <real_time_factor>1.0</real_time_factor>
</physics>



In [19]:
# Exporting this world configuration to a file allows running the 
# configured physics engine in Gazebo
sdf = physics.to_sdf('sdf')
print(sdf)

<sdf version="1.6">
  <world name="default">
    <gravity>0 0 -9.8</gravity>
    <include>
      <uri>model://ground_plane</uri>
    </include>
    <include>
      <uri>model://sun</uri>
    </include>
    <physics default="1" name="custom_ode" type="ode">
      <max_contacts>10</max_contacts>
      <ode>
        <constraints>
          <cfm>0.0</cfm>
          <erp>0.2</erp>
          <contact_surface_layer>0.001</contact_surface_layer>
          <contact_max_correcting_vel>100.0</contact_max_correcting_vel>
        </constraints>
        <solver>
          <min_step_size>0.0001</min_step_size>
          <precon_iters>0</precon_iters>
          <sor>1.5</sor>
          <use_dynamic_moi_rescaling>0</use_dynamic_moi_rescaling>
          <friction_model>box_model</friction_model>
          <type>quick</type>
          <iters>50</iters>
        </solver>
      </ode>
      <max_step_size>0.005</max_step_size>
      <real_time_update_rate>1000.0</real_time_update_rate>
      <real_time_fac

In [20]:
# Export to a .world file
world_filename = '/tmp/physics_ode.world'
sdf.export_xml(filename=world_filename)

In [21]:
# Remove old gazebo task
server.create_simulation('ode')
simulation = server.get_simulation('ode')
# Create new task
simulation.create_gazebo_task(
    name='gazebo',
    world=world_filename,
    gui=True,
    physics=physics.engine,
    paused=False,
    required=True,
    process_timeout=100)
# A task named 'gazebo' the added to the tasks list
print(simulation.get_task_list())
# But it is still not running
print('Is Gazebo running: {}'.format(simulation.is_task_running('gazebo')))
# Run Gazebo
simulation.run_all_tasks()

2020-09-11 14:26:14,719 | WARNING | __init__ | Error testing Gazebo server, message=Command '['rostopic', 'list']' returned non-zero exit status 1
['gazebo']
Is Gazebo running: False
2020-09-11 14:26:14,979 | WARNING | __init__ | Error testing roscore, message=Command '['rostopic', 'list']' returned non-zero exit status 1
2020-09-11 14:26:15,219 | WARNING | __init__ | Error testing roscore, message=Command '['rostopic', 'list']' returned non-zero exit status 1
2020-09-11 14:26:15,441 | WARNING | __init__ | Error testing roscore, message=Command '['rostopic', 'list']' returned non-zero exit status 1


In [22]:
# Check if the parameters were initialized correctly
gazebo_proxy = simulation.get_gazebo_proxy()

# It is important to note that the default get_physics_properties service from Gazebo
# returns only the global and the ODE engine parameters
print(gazebo_proxy.get_physics_properties())

time_step: 0.005
pause: False
max_update_rate: 1000.0
gravity: 
  x: 0.0
  y: 0.0
  z: -9.8
ode_config: 
  auto_disable_bodies: False
  sor_pgs_precon_iters: 0
  sor_pgs_iters: 50
  sor_pgs_w: 1.5
  sor_pgs_rms_error_tol: 0.0
  contact_surface_layer: 0.001
  contact_max_correcting_vel: 100.0
  cfm: 0.0
  erp: 0.2
  max_contacts: 10
success: True
status_message: "GetPhysicsProperties: got properties"


In [23]:
simulation.wait()
simulation.kill_all_tasks()

2020-09-11 14:27:56,224 | WARNING | task | Task <gazebo> - Killing process tree...
2020-09-11 14:27:56,252 | WARNING | task | Sending SIGINT to child process id=14667
2020-09-11 14:27:56,254 | WARNING | task | Child process 14667 still running
2020-09-11 14:27:56,256 | WARNING | task | Sending SIGINT to child process id=14690
2020-09-11 14:27:56,258 | WARNING | task | Child process 14690 still running
2020-09-11 14:27:56,259 | WARNING | task | Sending SIGINT to child process id=14685
2020-09-11 14:27:56,260 | WARNING | task | Child process 14685 still running
2020-09-11 14:27:56,261 | WARNING | task | Sending SIGINT to child process id=14756
2020-09-11 14:27:56,262 | WARNING | task | Child process 14756 still running
2020-09-11 14:27:56,263 | WARNING | task | Sending SIGINT to child process id=14759
2020-09-11 14:27:56,264 | WARNING | task | Child process 14759 still running
2020-09-11 14:27:56,265 | WARNING | task | Sending SIGINT to child process id=14666
2020-09-11 14:27:56,267 | WA

## Bullet

In [33]:
from pcg_gazebo.simulation.physics import Bullet
physics = Bullet()

# Iterate through all parameters
for name in physics.get_parameter_names():
    print('{}: {}'.format(name, physics.get_parameter(name)))

min_step_size: 0.0001
sor: 1.3
default: False
split_impulse: True
max_contacts: 20
name: default_physics
engine: bullet
real_time_update_rate: 1000
cfm: 0
iters: 50
contact_surface_layer: 0.001
split_impulse_penetration_threshold: -0.01
erp: 0.2
type: sequential_impulse
real_time_factor: 1
max_step_size: 0.001


In [34]:
# The description of the parameters is also available
for name in physics.get_parameter_names():
    physics.print_description(name)

min_step_size
  Description: The time duration which advances with each iteration of the dynamics engine, this has to be no bigger than max_step_size under physics block. If left unspecified, min_step_size defaults to max_step_size
  Current value: 0.0001
sor
  Description: Set the successive over-relaxation parameter.
  Current value: 1.3
default
  Description: If true, this physics element is set as the default physics profile for the world. If multiple default physics elements exist, the first element marked as default is chosen. If no default physics element exists, the first physics element is chosen.
  Current value: False
split_impulse
  Description: Similar to ODE max_vel implementation
  Current value: True
max_contacts
  Description: Maximum number of contactsallowed between two entities. This value can be over ridden by a max_contacts element in a collision element
  Current value: 20
name
  Description: The name of this set of physics parameters
  Current value: default_phy

In [35]:
# It is also possible to generate the SDF files.
# The SDF object can also be retrieved and altered if necessary
print(physics.to_sdf('physics'))

<physics default="1" name="default_physics" type="bullet">
  <max_contacts>20</max_contacts>
  <bullet>
    <constraints>
      <cfm>0.0</cfm>
      <erp>0.2</erp>
      <contact_surface_layer>0.001</contact_surface_layer>
      <split_impulse_penetration_threshold>-0.01</split_impulse_penetration_threshold>
      <split_impulse>1</split_impulse>
    </constraints>
    <solver>
      <min_step_size>0.0001</min_step_size>
      <type>sequential_impulse</type>
      <sor>1.3</sor>
      <iters>50</iters>
    </solver>
  </bullet>
  <real_time_update_rate>1000.0</real_time_update_rate>
  <max_step_size>0.001</max_step_size>
  <real_time_factor>1.0</real_time_factor>
</physics>



In [36]:
print(physics.to_sdf('world'))

<world name="default">
  <gravity>0 0 -9.8</gravity>
  <include>
    <uri>model://ground_plane</uri>
  </include>
  <include>
    <uri>model://sun</uri>
  </include>
  <physics default="1" name="default_physics" type="bullet">
    <max_contacts>20</max_contacts>
    <bullet>
      <constraints>
        <cfm>0.0</cfm>
        <erp>0.2</erp>
        <contact_surface_layer>0.001</contact_surface_layer>
        <split_impulse_penetration_threshold>-0.01</split_impulse_penetration_threshold>
        <split_impulse>1</split_impulse>
      </constraints>
      <solver>
        <min_step_size>0.0001</min_step_size>
        <type>sequential_impulse</type>
        <sor>1.3</sor>
        <iters>50</iters>
      </solver>
    </bullet>
    <real_time_factor>1.0</real_time_factor>
    <max_step_size>0.001</max_step_size>
    <real_time_update_rate>1000.0</real_time_update_rate>
  </physics>
  <atmosphere type="adiabatic"/>
  <wind>
    <linear_velocity>0 0 0</linear_velocity>
  </wind>
</world>



In [37]:
# Let's change some parameters
physics.max_step_size = 0.005
physics.cfm = 0.01
physics.sor = 1.5
physics.max_contacts = 5
physics.name = 'custom_bullet'
physics.real_time_update_rate = 500
print(physics.to_sdf())

<physics default="1" name="custom_bullet" type="bullet">
  <max_contacts>5</max_contacts>
  <bullet>
    <constraints>
      <cfm>0.01</cfm>
      <erp>0.2</erp>
      <contact_surface_layer>0.001</contact_surface_layer>
      <split_impulse_penetration_threshold>-0.01</split_impulse_penetration_threshold>
      <split_impulse>1</split_impulse>
    </constraints>
    <solver>
      <min_step_size>0.0001</min_step_size>
      <type>sequential_impulse</type>
      <sor>1.5</sor>
      <iters>50</iters>
    </solver>
  </bullet>
  <real_time_update_rate>500.0</real_time_update_rate>
  <max_step_size>0.005</max_step_size>
  <real_time_factor>1.0</real_time_factor>
</physics>



In [38]:
# Exporting this world configuration to a file allows running the 
# configured physics engine in Gazebo
sdf = physics.to_sdf('sdf')
print(sdf)

<sdf version="1.6">
  <world name="default">
    <gravity>0 0 -9.8</gravity>
    <include>
      <uri>model://ground_plane</uri>
    </include>
    <include>
      <uri>model://sun</uri>
    </include>
    <physics default="1" name="custom_bullet" type="bullet">
      <max_contacts>5</max_contacts>
      <bullet>
        <constraints>
          <cfm>0.01</cfm>
          <erp>0.2</erp>
          <contact_surface_layer>0.001</contact_surface_layer>
          <split_impulse_penetration_threshold>-0.01</split_impulse_penetration_threshold>
          <split_impulse>1</split_impulse>
        </constraints>
        <solver>
          <min_step_size>0.0001</min_step_size>
          <type>sequential_impulse</type>
          <sor>1.5</sor>
          <iters>50</iters>
        </solver>
      </bullet>
      <real_time_update_rate>500.0</real_time_update_rate>
      <max_step_size>0.005</max_step_size>
      <real_time_factor>1.0</real_time_factor>
    </physics>
    <atmosphere type="adiabatic"/>

In [39]:
# Export to a .world file
world_filename = '/tmp/physics_bullet.world'
sdf.export_xml(filename=world_filename)

In [40]:
# Remove old gazebo task
server.create_simulation('bullet')
simulation = server.get_simulation('bullet')
# Create new task
simulation.create_gazebo_task(
    name='gazebo',
    world=world_filename,
    gui=True,
    physics=physics.engine,
    paused=False,
    required=True,
    process_timeout=100)
# A task named 'gazebo' the added to the tasks list
print(simulation.get_task_list())
# But it is still not running
print('Is Gazebo running: {}'.format(simulation.is_task_running('gazebo')))
# Run Gazebo
simulation.run_all_tasks()

# The get_physics_properties service does not support bullet parameters yet

2020-09-11 14:32:20,204 | ERROR | server | Simulation with label bullet already exists
2020-09-11 14:32:20,206 | ERROR | process_manager | Task gazebo already exists
['roscore', 'gazebo']
Is Gazebo running: False
2020-09-11 14:32:20,447 | WARNING | __init__ | Error testing roscore, message=Command '['rostopic', 'list']' returned non-zero exit status 1
2020-09-11 14:32:20,698 | WARNING | __init__ | Error testing roscore, message=Command '['rostopic', 'list']' returned non-zero exit status 1


In [ ]:
simulation.wait()
simulation.kill_all_tasks()

## Simbody

In [30]:
from pcg_gazebo.simulation.physics import Simbody
physics = Simbody()

# Iterate through all parameters
for name in physics.get_parameter_names():
    print('{}: {}'.format(name, physics.get_parameter(name)))

max_step_size: 0.001
real_time_factor: 1
real_time_update_rate: 1000
max_contacts: 20
engine: simbody
name: default_physics
default: False
min_step_size: 0.0001
accuracy: 0.001
max_transient_velocity: 0.01
stiffness: 100000000.0
dissipation: 100
plastic_coef_restitution: 0.5
plastic_impact_velocity: 0.5
static_friction: 0.9
dynamic_friction: 0.9
viscous_friction: 0
override_impact_capture_velocity: 0.001
override_stiction_transition_velocity: 0.001


In [31]:
# The description of the parameters is also available
for name in physics.get_parameter_names():
    physics.print_description(name)

max_step_size
  Description: Maximum time step size at which every system in simulation can interact with the states of the world
  Current value: 0.001
real_time_factor
  Description: Target simulation speedupfactor, defined by ratio of simulation time to real-time
  Current value: 1
real_time_update_rate
  Description: Rate at which to update the physics engine (UpdatePhysics calls per real-time second)
  Current value: 1000
max_contacts
  Description: Maximum number of contactsallowed between two entities. This value can be over ridden by a max_contacts element in a collision element
  Current value: 20
engine
  Description: The type of the dynamics engine. Current options are ode, bullet, simbody and dart. Defaults to ode if left unspecified.
  Current value: simbody
name
  Description: The name of this set of physics parameters
  Current value: default_physics
default
  Description: If true, this physics element is set as the default physics profile for the world. If multiple defa

In [32]:
# It is also possible to generate the SDF files.
# The SDF object can also be retrieved and altered if necessary
print(physics.to_sdf('physics'))

<physics name="default_physics" default="1" type="simbody">
  <max_step_size>0.001</max_step_size>
  <real_time_factor>1.0</real_time_factor>
  <real_time_update_rate>1000.0</real_time_update_rate>
  <max_contacts>20</max_contacts>
  <simbody>
    <min_step_size>0.0001</min_step_size>
    <accuracy>0.001</accuracy>
    <max_transient_velocity>0.01</max_transient_velocity>
    <contact>
      <stiffness>100000000.0</stiffness>
      <dissipation>100.0</dissipation>
      <plastic_coef_restitution>0.5</plastic_coef_restitution>
      <plastic_impact_velocity>0.5</plastic_impact_velocity>
      <static_friction>0.9</static_friction>
      <dynamic_friction>0.9</dynamic_friction>
      <viscous_friction>0.0</viscous_friction>
      <override_impact_capture_velocity>0.001</override_impact_capture_velocity>
      <override_stiction_transition_velocity>0.001</override_stiction_transition_velocity>
    </contact>
  </simbody>
</physics>



In [33]:
print(physics.to_sdf('world'))

<world name="default">
  <physics name="default_physics" default="1" type="simbody">
    <max_step_size>0.001</max_step_size>
    <real_time_factor>1.0</real_time_factor>
    <real_time_update_rate>1000.0</real_time_update_rate>
    <max_contacts>20</max_contacts>
    <simbody>
      <min_step_size>0.0001</min_step_size>
      <accuracy>0.001</accuracy>
      <max_transient_velocity>0.01</max_transient_velocity>
      <contact>
        <stiffness>100000000.0</stiffness>
        <dissipation>100.0</dissipation>
        <plastic_coef_restitution>0.5</plastic_coef_restitution>
        <plastic_impact_velocity>0.5</plastic_impact_velocity>
        <static_friction>0.9</static_friction>
        <dynamic_friction>0.9</dynamic_friction>
        <viscous_friction>0.0</viscous_friction>
        <override_impact_capture_velocity>0.001</override_impact_capture_velocity>
        <override_stiction_transition_velocity>0.001</override_stiction_transition_velocity>
      </contact>
    </simbody>
  <

In [34]:
# Let's change some parameters
physics.max_step_size = 0.005
physics.max_contacts = 8
physics.name = 'custom_simbody'
physics.static_friction = 1.0
physics.real_time_update_rate = 500
print(physics.to_sdf())

<physics name="custom_simbody" default="1" type="simbody">
  <max_step_size>0.005</max_step_size>
  <real_time_factor>1.0</real_time_factor>
  <real_time_update_rate>500.0</real_time_update_rate>
  <max_contacts>8</max_contacts>
  <simbody>
    <min_step_size>0.0001</min_step_size>
    <accuracy>0.001</accuracy>
    <max_transient_velocity>0.01</max_transient_velocity>
    <contact>
      <stiffness>100000000.0</stiffness>
      <dissipation>100.0</dissipation>
      <plastic_coef_restitution>0.5</plastic_coef_restitution>
      <plastic_impact_velocity>0.5</plastic_impact_velocity>
      <static_friction>1.0</static_friction>
      <dynamic_friction>0.9</dynamic_friction>
      <viscous_friction>0.0</viscous_friction>
      <override_impact_capture_velocity>0.001</override_impact_capture_velocity>
      <override_stiction_transition_velocity>0.001</override_stiction_transition_velocity>
    </contact>
  </simbody>
</physics>



In [35]:
# Exporting this world configuration to a file allows running the 
# configured physics engine in Gazebo
sdf = physics.to_sdf('sdf')
print(sdf)

<sdf version="1.6">
  <world name="default">
    <physics name="custom_simbody" default="1" type="simbody">
      <max_step_size>0.005</max_step_size>
      <real_time_factor>1.0</real_time_factor>
      <real_time_update_rate>500.0</real_time_update_rate>
      <max_contacts>8</max_contacts>
      <simbody>
        <min_step_size>0.0001</min_step_size>
        <accuracy>0.001</accuracy>
        <max_transient_velocity>0.01</max_transient_velocity>
        <contact>
          <stiffness>100000000.0</stiffness>
          <dissipation>100.0</dissipation>
          <plastic_coef_restitution>0.5</plastic_coef_restitution>
          <plastic_impact_velocity>0.5</plastic_impact_velocity>
          <static_friction>1.0</static_friction>
          <dynamic_friction>0.9</dynamic_friction>
          <viscous_friction>0.0</viscous_friction>
          <override_impact_capture_velocity>0.001</override_impact_capture_velocity>
          <override_stiction_transition_velocity>0.001</override_stiction

In [36]:
# Export to a .world file
world_filename = '/tmp/physics_simbody.world'
sdf.export_xml(filename=world_filename)

In [37]:
server.create_simulation('simbody')
simulation = server.get_simulation('simbody')
# Create new task
simulation.create_gazebo_task(
    name='gazebo',
    world=world_filename,
    gui=True,
    physics=physics.engine,
    paused=False,
    required=True,
    process_timeout=10)
# A task named 'gazebo' the added to the tasks list
print(simulation.get_task_list())
# But it is still not running
print('Is Gazebo running: {}'.format(simulation.is_task_running('gazebo')))
# Run Gazebo
simulation.run_all_tasks()

# The get_physics_properties service does not support simbody parameters yet

['gazebo']
Is Gazebo running: False


In [38]:
simulation.wait()
simulation.kill_all_tasks()